# RAG

- "RAG"는 "Retrieval-Augmented Generation"의 약자로, "검색-증강 생성"이라는 의미를 가지고 있습니다. 이는 자연어 처리(NLP) 및 기계 학습 분야, 특히 챗봇이나 질문-응답 시스템과 같은 고급 언어 모델을 구축하는 데 사용되는 기술입니다.

- RAG에 대한 간략한 개요는 다음과 같습니다:

검색과 생성의 결합: RAG는 NLP의 두 가지 주요 구성 요소인 정보 검색과 응답 생성을 결합합니다. 검색 부분은 관련 정보를 찾기 위해 대규모 데이터베이스나 문서 컬렉션을 검색하는 과정을 포함합니다. 생성 부분은 검색된 정보를 바탕으로 일관되고 맥락적으로 적절한 텍스트를 생성하는 과정입니다.


작동 방식: RAG 시스템에서 질문이나 프롬프트가 주어지면 모델은 먼저 질문에 대한 답변을 제공하는 데 유용한 정보를 포함할 수 있는 관련 문서나 텍스트를 검색합니다. 그런 다음 이 검색된 정보를 생성 모델에 공급하여 일관된 응답을 합성합니다.


- 장점: RAG의 주요 장점은 모델이 외부 지식을 활용할 수 있게 하여 보다 정확하고 상세하며 맥락적으로 관련된 답변을 제공할 수 있다는 것입니다. 이는 특정 지식이나 사실적 정보가 필요한 질문에 특히 유용합니다.


- 응용 분야: RAG는 챗봇, 질문-응답 시스템 및 정확하고 맥락적으로 관련된 정보를 제공하는 것이 중요한 다른 AI 도구와 같은 다양한 응용 분야에 사용됩니다. 특히 모델이 다양한 주제와 데이터를 기반으로 이해하고 응답을 생성해야 하는 상황에서 유용합니다.
개발 및 사용: AI 및 기계 학습 커뮤니티에서 RAG는 다양한 연구 논문과 구현이 개발되고 있으며 주요 초점 중 하나입니다. 이는 학습된 정보뿐만 아니라 외부 소스에서 새롭고 관련된 정보를 통합하여 응답의 질과 관련성을 향상시키는 더 정교한 AI 시스템으로 나아가는 단계를 나타냅니다.

In [ ]:
!pip install chromadb

## Retrieval - RAG의 첫 번째 단계

### Data Loader 

- Loader는 소스에서 데이터를 추출하고 랭체인에 가져다 주는 코드 


### Splitters

- 문서에서 필요한 부분만 찾을 때 ! 

- 두 가지 방법이 있음. 


- from_tiktoken_encoder : 텍스트의 길이를 모델이 카운트 하는 방법 (모델이 자연어를 바라보는 방법. 토큰)


#### Embedding : 사람이 읽는 텍스트를 컴퓨터가 이해할 수 있는 숫자들로 변환하는 작업. 

- 단어를 표현하는 벡터로 만든다! 
- 단어를 차원들로 표현할 때 좋은 점은 다른 단어를 얻기 위해 가진 단어를 가지고 연산을 할 수 있음. 
- 단어를 연관시키고 싶다면 ? 
    - https://turbomaze.github.io/word2vecjson/ 이런 웹사이트에서 단어를 입력하면 연관된 단어를 찾거나 벡터 연산을 통해 비슷한 단어를 찾아준다. (참고)
    - https://www.youtube.com/watch?v=2eWuYf-aZE4 스포티파이에서 LLM을 어떻게 활용하나 ? 

- Embedding을 매번 코드를 실행할 때마다 하는 것은 좋은게 아니다. 한 번 document를 embed 하면 문서가 바뀔 때까지 embeddings도 바뀌지 않아야한다. 

- **Vector Store** : 일종의 데이터 벡스. 벡터 공간에서 검색을 할 수 있게 해준다. 벡터들을 만들고 나서 그것들을 캐싱해주고 vector stroe에 그 벡터들을 넣어주면, 우리가 검색을 할 수 있다. 
    - 다양한 vector store를 사용할 수 있다. (유료 무료 버전이 있다.)
    - Chroma 사용. 
        - Chroma에는 분할된(Splitted) 문서와 openAI embeddings model을 전달해야 한다. 
        - cache 폴더에 처음 embedding한 값들이 저장된다.


- langsmith : 시각화 툴. 베타 서비스 중이라서 가입후 웨이팅 해야함. (https://smith.langchain.com/o/3a7fde8e-2b57-5c01-aae7-c830f9fc466c/settings) 

- API key 생성 후 .env 파잃에 OPEN_API_KEY 등록.
- 데이터의 흐름을 파악하고 이해할 수 있다. 

In [ ]:
from typing import Text
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)# 문장이나 문단 단위로 자를 수 있다 !

loader = UnstructuredFileLoader("./files/chapter1.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

In [ ]:
# Embedding 예시

from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

vector = embedder.embed_documents([
    "hi",
    "how",
    "are",
    "you longer sentences because"
])

print(len(vector), len(vector[0])) # 단어의 개수, 벡터의 차원


## RetirevalQA

"RetrievalQA" 기능은 특히 질문에 대한 답변을 찾기 위해 외부의 문서나 데이터베이스에서 정보를 검색(retrieval)하는 과정을 자동화하는 데 중점을 두고 있습니다.

RetrievalQA은 개념이 모호해서 공식문서에 설명이 없다

- 공식문서에는 LangChain Expression Language(LCEL)을 사용할 것을 권장합니다. 
- LCEL은 어렵다 ! 

- Stuff : 우리가 찾은 document들로 prompt를 stuff(채우기) 하는데 사용한다는 뜻.

- retriver는 class의 interface 입니다. document를 많은 장소로부터 Retrieve(선별하여 가져오기)할 수 있습니다. vector store말고 다른 곳에서도. (database, cloud도 가능)

In [ ]:
from typing import Text
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)# 문장이나 문단 단위로 자를 수 있다 !

loader = UnstructuredFileLoader("./files/chapter1.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever = vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansions")

# Recap

- file load를 해보자! unstructuredFileLoader에 대해서 사용. ( 다양한 file format을 load할 수 있음.) 

- file을 분할(split)하는 방법도 배웠음. 아주 긴 text를 다루기 위해서. text를 작은 document로 나눠보자!

- 거대한 단일 문서보다 작은 여러개를 LLM에게 전달할 때 검색 기능이 더 좋아짐 ( 작업이 쉬워지고 응답도 빨라지고 비용도 줄어듦 )

- Embedding은 text에 의미별로 적절한 점수를 부여해서 vector 형태로 표현한 것 ( OpenAIEmbedding model을 사용함)

- CacheBackedEmbedding을 사용하여 만들어진 Embedding을 cache(저장)했음. Embedding은 공짜가 아니기 때문. 
 - CacheBackedEmbedding class 사용. embeddings model을 전달하고, 데이터가 cache될 directory를 지정했음. 
 - vector store 호출. FAISS 사용. from_document는 document 별로 embedding 작업 후 결과를 저장한 vector store를 반환하는데 이를 이용해 documet 검색도 하고 연관성 높은 document를 찾기도 함. 

- RetrievalQA 는 LCEL을 사용해서 만들 것!

### stuff LCEL Chain    

chain.invoke()를 실행하면 string이 retriever에 전달 될 것입니다. retriever은 document들의 list를 반환할거고 그 document는 context 값으로 입력됩니다.
그리고 아래의 질문도 pompt template의 question으로 전달되어야 합니다. 

```python
retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)
chain = {"context":retriver, "question": RunnablePassthrough(), "extra": RunnablePassthrough()} | prompt | llm 
```

RunnablePassthrough() 클래스는 chain.invoke("something")의 something을 말그대로 통과하게 해줄 것입니다. 그러면 "question"에 "something"이 가는거죠. 

결국 "context" : [Doc] , "question" : "Describe Victory Mansions" 를 하는 것과 똑같음


#### Map Reduce LCEL Chain 

map reduce 의 원리: 우선 document의 list를 얻어야 한다. 그 다음으로 list 내부의 모든 document를 위한 prompt를 만들어야함. 그 prompt는 LLM에게 전달. 
prompt는 *'이 document'를 읽고, 사용자의 질문에 답변하기에 적절한 정보가 있는지 확인하시오.'* 
이를 전달받은 LLM은 답변을 출력함. 그리고 LLM으로부터 받은 response 들을 취합해 하나의 document를 만들어냄. 
최종 document는 LLM을 위한 Prompt로 전달. 

example : '빅토리 멘션을 묘사해주세요' 질문 -> 질문은 우선 retriever로 전달된다. 그러면 retriever은 빅토리 멘션을 묘사하는 것과 관련이 있는 document list를 반환한다. 그 list의 모든 document에 대한 prompt를 만들고, 그걸 LLM에 전달. LLM은 "이 문서를 읽고 질문에 답하는데에 관련이있는 중요한 정보를 추출하세요" 이런 내용을 담고있음. 
그 작업이 list의 모든 document에서 수행됩니다. 만약 5개의 document가 있다면, LLM에게 5번 질문해서 5개의 응답(response)를 받을 것입니다. 그리고 그 응답들을 전부 묶어서 하나의 긴 document를 생성합니다. 그 최종 document 하나가 prompt에 입력되어 LLM에게 전달됩니다. prompt의 내용은 "이것은 질문과 관련이 잇는 정보들입니다. 이를 사용하여 대답해주세요" 

stuff와 map reduce 방식 중 어떤 방법을 선택해야할까? 
내가 원하는 prompt의 크기와 검색할 document의 수에 따라 달라집니다. 만약 retriever가 검색 결과로 천 개 이상의 document를 반환한다면, stuff는 사용할 수 없습니다. 왜냐하면 stuff의 prompt에 그 document들을 모두 넣을 수 없기 때문이죠. 부적합합니다. 
document가 아주 많은 상황에 적합한건 map reduce 방식입니다.  

---

tempalte이 필요로하는 context를 만드는 방법. context에는 긴 document가 들어갑니다. LLM이 추출한 여러 다른 document들의 작은 부분들을 모아서 만들어집니다. 

map_chain은 마지막 chain 내부에서 호출(call)될 것입니다. 따라서 invoke 메서드가 실행될 때, map_chain이 context로 들어갈 것이고 그리고 RunnablePassthrough()에는 "Describe Victory Mensions"가 들어가겠죠? 그리고 실행 결과는 모두 final_prompt에 전달됩니다. 

map_chain은 document가 필요합니다. 그리고 document는 retriever를 사용해서 얻을 수 있습니다. 또 사용자의 질문 내용을 알아야 합니다. 그래야 LLM에게 요청할 수 있거든요.
각 document를 살펴보면서 사용자 질문에 대답하는데 필요한 정보가 담겨있는지 알아봐달라고 말이죠. 이제 두 개의 데이터가 필요하다는 것에 대한 설명이 되었습니다. 

map_chain에서 doc를 retrieve로 입력받습니다. retriever는 string 값을 입력받고 document들을 출력해줍니다. 
그리고 사용자가 보낸 질문(question)도 전달합니다. 왜냐하면 retriever가 우리에게 document를 제공하면 어떤 function을 실행할건데 거기서 document가 사용자의 질문(question)과 관련된 정보를 가졌는지 확인해야 하기 때문이죠. 

이제 list의 각 document별로 작업을 수행하고 그것들이 question에 대한 답변을 하는 데 필요한 관련 정보를 포함하는지 확인해봅니다. 
이 작업을 위한 map_docs() 함수 작성. inputs의 documents와 question을 통해 추출합니다. 이 function을 추출하기 위해 runnablelambda를 사용하는 것이죠.
이제 map_docs의 return 값에 대해 생각해봐야 합니다. 우리의 chain은 context 값을 얻기위해 map_chain을 실행할 것입니다. 
map_chain을 보면 documents와 사용자의 question을 입력받습니다. 그리고 map_chain의 목표는, 한 개의 string을 반환하는 것입니다. 
사용자의 question에 대한 답변 생성에 관련이 있는 정보를 포함한 document의 일부나 전체를 말합니다. map_docs의 반환값은 한 개의 string이 되어야 합니다. 
우리가 원하는 return 값은 inputs의 retriever로부터 반환받은 documents의 개별 요소마다 또 다른 chain을 실행하는 것입니다. 각각 따로 말이죠. 
또다른 chain을 통해 관련이있는 정보를 추출해낼 것입니다. 

또 다른 chain을 만들어야함 !! 
map_doc_chain을 만들어야함. map_doc_chain을 우선 만들어보자. 
해당 체인의 prompt에서 관련있는 text는 변경하지 말고, 그냥 반환해주었으면 한다고 합니다. 그냥 관련성이 있는지 확인하고 있다면 그대로 전달 부탁. 
이제 documents 내부의 각 document에 대해 이 chain(map_doc_chain)을 실행해줍니다. 그리고 각각의 response(응답)을 저장할 responses라는 list를 만들 것입니다. 
그리고 그것을 하나의 긴 string으로 변환할 것입니다. 한 개의 긴 document로 말이죠. 

아래의 함수의 동작을 확인하고 result의 list인 results를 전부 한 개의 document에 넣을 것입니다. 그렇게 하기위해서 join 메서드로 results의 아이템들을 하나의 string으로 합쳐줍니다.

```python
def map_docs(inputs):
    documents = inputs['documents']
    question = inputs['question']
    results = []
    for document in documents: 
        result = map_doc_chain.invoke({
            "context": document.page_content , 
            "question": question}).content
        results.append(result)
    results = "\n\n".join(results) 

map_chain = {"documents": retriever, "question": RunnablePassthrough()} | RunnableLambda(map_docs)
```

정리하자면, 위의 코드는 map_doc_chain의 invoke 결과로 만들어진 result의 content list를 만드는 것입니다. chain은 각 document의 page_content와 사용자 question을 입력받아 작업을 실행합니다. 
똑같은 아이디어를 사용하지만 코드를 조금 더 다듬어 보겠습니다. 

```python
def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in documents
    )
```


- RunnableLambda는 chain과 그 내부 어디에서든 function을 호출할 수 있도록 해줍니다. 

In [10]:
from typing import Text

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter1.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()


# "system" : 다음의 긴 document의 일부 중, 질문에 대한 답변을 생성하는 것과 관련이 있는 부분을 찾아주세요. 관련있는 부분을 찾는다면 해당 text를 그대로 반환해주세요.
map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm  

def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in documents
    )

map_chain = {"documents": retriever, "question": RunnablePassthrough()} | RunnableLambda(map_docs)
 
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": map_chain,  "question":RunnablePassthrough()} | final_prompt | llm

chain.invoke("Where does Winston go to work?")

AIMessage(content='Winston goes to work at the Ministry of Truth.')

In [7]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 39.8 MB/s eta 0:00:0000:0100:01
